In [5]:
$run object_tracker.py --video tracker01.mp4 --tracker kcf

SyntaxError: invalid syntax (<ipython-input-5-7b51ff5457a9>, line 1)

In [9]:
import imutils
import argparse
import time
import cv2
from imutils.video import VideoStream
from imutils.video import FPS

ap=argparse.ArgumentParser()
ap.add_argument("-v","--video",type=str,help="path to input video file")
ap.add_argument("-t","--tracker",type=str,default="kcf",help="path to input video file")
args=vars(ap.parse_args())

(major,minor)=cv2.__version__.split(".")[:2]
if int(minor)<3:
    tracker=cv2.Tracker_create(args["tracker"].upper())
else:
    OPENCV_OBJECT_TRACKERS ={
        "csrt":cv2.TrackerCSRT_create,
        "kcf":cv2.TrackerKCF_create,
        "boosting":cv2.TrackerBoosting_create,
        "mil":cv2.TrackerMIL_create,
        "tld":cv2.TrackerTLD_create,
        "medianflow":cv2.TrackerMedianFlow_create,
        "mosse":cv2.TrackerMOSSE_create,
    }
    tracker=OPENCV_OBJECT_TRACKERS[args["tracker"]]()

initBB=None
if not args.get("video",False):
    print("[INFO] starting video stream....")
    vs=VideoStream(src=0).start()
    time.sleep(1.0)
else:
    vs=cv2.VideoCapture(args["video"])

fps=None

while True:
    frame=vs.read()
    frame=frame[1] if args.get("video",False) else frame

    if frame in None:
        break
    frame=imutils.resize(frame,width=500)
    (H,W)=frame.shape[:2]

    if initBB is not None:
        (success,box)=tracker.update(frame)
        if success:
            (x,y,w,h)=[int(v) for v in box]
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)

        fps.update()
        fps.stop()

        info=[
            ("Tracker",args["tracker"]),
            ("Success","Yes" if success else "No"),
            ("FPS","{:.2f}".format(fps.fps())),
        ]
        for (i,(k,v)) in enumerate(info):
            text="{}:{}".format(k,v)
            cv2.putText(frame,text,(10,H-((i*20)+20)),cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,0,255),2)
            cv2.imshow("Frame",frame)
            key=cv2.waitKey(1) & 0xFF
            if key==ord("s"):
                initBB=cv2.selectROI("Frame",frame,fromCenter=False,showCrosshair=True)
                tracker.init(frame,initBB)
                fps=FPS().start()
            elif key==ord("q"):
                break

            if not args.get("video",False):
                vs.stop()
            else:
                vs.release()

            cv2.destroyAllWindows()


AttributeError: module 'cv2' has no attribute 'INTER_AREA'

In [1]:
import cv2
import numpy as np
videocap=cv2.VideoCapture('traffic.flv')
firstframe=None
while True:
	ret,frame=videocap.read()
	if not ret:
		break
	pp=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)      #将RGB转换为GSV
	if firstframe is None:                     #前面已经设置为空
		firstframe=pp
		continue
	frameDelta=cv2.absdiff(firstframe,pp)      #帧差法，将firstframe和gray的差值输出到frameDelta中
	thresh=cv2.threshold(frameDelta,25,255,cv2.THRESH_BINARY)[1]   #图像二值化（使得图像明显黑白化）
	thresh=cv2.dilate(thresh,None,iterations=2)              
	x,y,w,h=cv2.boundingRect(thresh)       #确定目标大小
	frame=cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),2)   #在frame上画出矩形
	cv2.imshow("frame",frame)    #原视频文件
	cv2.imshow("Thresh",thresh)
	cv2.imshow("frameDelta",frameDelta)
	key=cv2.waitKey(50)&0xFF
	if key==ord("q"):
		break
videocap.release()


cv2.destroyAllWindows()
